**kết nối với drive cá nhân**

Kết nối với drive để có thể lưu model và checkpoint khi finetune để dễ dàng truy cập và sử dụng lại cho lần kế tiếp

In [1]:
#kết nối với drive cá nhân
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#1 Data Preparation


In [2]:
# Cài đặt các thư viện
!pip install transformers datasets torch torchvision pandas numpy tqdm faiss-cpu gradio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

**Import Libraries**

In [3]:
from google.colab import drive
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPModel, CLIPProcessor
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
from PIL import Image
import os
import json
import numpy as np
from sklearn.cluster import KMeans
from pathlib import Path
import gradio as gr
from pyngrok import ngrok
from huggingface_hub import HfApi, Repository
import torch.nn.functional as F
import urllib.request

##Download dataset from drive


In [ ]:
#Deepfashion.zip
#https://drive.google.com/file/d/1OVE0M_sWtRVPvpDAlWAEhbpCpcXdLajW/view?usp=sharing
#https://drive.google.com/file/d/1tvw63gM7uVSI_uSaBosoBfWZ6HYRHgIv/view?usp=sharing
#!gdown '1tvw63gM7uVSI_uSaBosoBfWZ6HYRHgIv'


Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1tvw63gM7uVSI_uSaBosoBfWZ6HYRHgIv

but Gdown can't. Please check connections and permissions.


**Thay đổi, vì mỗi lần phải kết nối lại và chạy code cell để finetune nên tới một lúc nào đó drive sẽ chặn không cho download dataset nữa nên chuyển sang lưu trữ dataset trên huggingface sẽ không bị giới hạn, upload dataset lên huggingface và tải vào google colab để sử dụng.**

##Download dataset from Huggingface dataset

In [4]:
!wget -O DeepFashion.zip https://huggingface.co/datasets/anhquanlam/LQAQ_DeepFashion_multimodal/resolve/main/DeepFashion2.zip

#!curl -L -o DeepFashion.zip https://huggingface.co/datasets/anhquanlam/LQAQ_DeepFashion_multimodal/resolve/main/DeepFashion2.zip


--2025-07-11 09:56:51--  https://huggingface.co/datasets/anhquanlam/LQAQ_DeepFashion_multimodal/resolve/main/DeepFashion2.zip
Resolving huggingface.co (huggingface.co)... 3.165.102.128, 3.165.102.58, 3.165.102.22, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a3/b9/a3b9072278333650f3747985449803cd4cb4f4e58e22fb69db6596bd83ac5469/8e130868591b42b735823e11864f2eb41c0cc08ce6d3ecdc32d445cd8a3e980c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27DeepFashion2.zip%3B+filename%3D%22DeepFashion2.zip%22%3B&response-content-type=application%2Fzip&Expires=1752231412&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjIzMTQxMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2EzL2I5L2EzYjkwNzIyNzgzMzM2NTBmMzc0Nzk4NTQ0OTgwM2NkNGNiNGY0ZTU4ZTIyZmI2OWRiNjU5NmJkODNhYzU0NjkvOGUxMzA4Njg1OTFiNDJiNzM1ODIzZTExODY0Z

In [ ]:
# Tải DeepFashion.zip từ Hugging Face Datasets
#url = "https://huggingface.co/datasets/anhquanlam/LQAQ_DeepFashion_multimodal/resolve/main/DeepFashion2.zip"
#output_zip = "DeepFashion2.zip"
# Tải file zip
#if not os.path.exists(output_zip):
    #print("Downloading dataset from Hugging Face...")
    #urllib.request.urlretrieve(url, output_zip)
    #print("Download complete.")

Download complete.


##Unzip dataset



In [5]:
# Đường dẫn đến thư mục chứa dataset
data_dir = '/content/deepfashion_multimodal'
os.makedirs(data_dir, exist_ok=True)

# Đường dẫn đến file zip vừa tải
#zip_path = '/content/DeepFashion.zip'
zip_path = '/content/DeepFashion.zip'
# Giải nén file với tùy chọn -q (quiet) để giảm output
!unzip -q "{zip_path}" -d "{data_dir}"
print(f"Extracted {zip_path} to {data_dir}")

Extracted /content/DeepFashion.zip to /content/deepfashion_multimodal


##Inspecting the directory

In [6]:
# Đường dẫn đến thư mục chứa dataset
extracted_dir = os.path.join(data_dir, 'DeepFashion')

# Kiểm tra nội dung thư mục DeepFashion
print("Contents of the DeepFashion directory:")
!ls -la "{extracted_dir}"

# Kiểm tra số lượng file trong thư mục images
images_dir = os.path.join(extracted_dir, 'images')
if os.path.exists(images_dir):
    num_images = len([f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))])
    print(f"Number of images in {images_dir}: {num_images}")
else:
    print(f"Directory {images_dir} not found.")

# Kiểm tra Captions.json
captions_json_path = os.path.join(extracted_dir, 'captions.json')
if os.path.exists(captions_json_path):
    print(f"captions.json found. Size: {os.path.getsize(captions_json_path)} bytes")
    with open(captions_json_path, 'r') as f:
        captions_data = json.load(f)
        print(f"Sample from captions.json: {list(captions_data.items())[:2]}")
else:
    print("captions.json not found.")

Contents of the DeepFashion directory:
total 15040
drwxrwxrwx 4 root root     4096 Jul  9 13:05 .
drwxr-xr-x 3 root root     4096 Jul 11 09:58 ..
-rw-rw-rw- 1 root root 11968792 Jul  3 16:01 captions.json
drwxrwxrwx 2 root root     4096 Jul  9 13:05 embed_data
drwxrwxrwx 2 root root  3407872 Jul  3 16:27 images
Number of images in /content/deepfashion_multimodal/DeepFashion/images: 44096
captions.json found. Size: 11968792 bytes
Sample from captions.json: [('MEN-Denim-id_00000080-01_7_additional.jpg', 'The lower clothing is of long length. The fabric is cotton and it has plaid patterns.'), ('MEN-Denim-id_00000089-01_7_additional.jpg', 'His tank top has sleeves cut off, cotton fabric and pure color patterns. The neckline of it is round. The pants this man wears is of long length. The pants are with cotton fabric and pure color patterns.')]


#2 Tạo Dataset

In [7]:
# Đường dẫn đến thư mục chứa dataset
data_dir = '/content/deepfashion_multimodal'
extracted_dir = os.path.join(data_dir, 'DeepFashion')
image_dir = os.path.join(extracted_dir, 'images')
captions_json_path = os.path.join(extracted_dir, 'captions.json')


In [8]:
# Khởi tạo processor từ clip-vit-base-patch16
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
# Đọc captions.json
with open(captions_json_path, 'r') as f:
    captions_data = json.load(f)

# Chuyển đổi sang DataFrame
caption_df = pd.DataFrame(list(captions_data.items()), columns=['image_name', 'caption'])

In [10]:
# Tính độ dài token của mỗi caption
def get_token_length(caption):
    inputs = processor.tokenizer(caption, return_tensors="pt", truncation=False)
    return inputs['input_ids'].shape[1]

caption_df['token_length'] = caption_df['caption'].apply(get_token_length)

# Thống kê độ dài và tìm caption dài nhất
max_length = caption_df['token_length'].max()
min_length = caption_df['token_length'].min()
mean_length = caption_df['token_length'].mean()
num_longer_than_77 = (caption_df['token_length'] > 77).sum()

print(f"Max token length: {max_length}")
print(f"Min token length: {min_length}")
print(f"Mean token length: {mean_length}")
print(f"Number of captions longer than 77 tokens: {num_longer_than_77}")

# Hiển thị 5 caption dài nhất
longest_captions = caption_df.sort_values(by='token_length', ascending=False).head(5)
print("\nTop 5 longest captions:")
print(longest_captions)

Token indices sequence length is longer than the specified maximum sequence length for this model (81 > 77). Running this sequence through the model will result in indexing errors


Max token length: 112
Min token length: 2
Mean token length: 50.402947536667924
Number of captions longer than 77 tokens: 1744

Top 5 longest captions:
                                            image_name  \
13691        WOMEN-Cardigans-id_00006470-04_4_full.jpg   
13692  WOMEN-Cardigans-id_00006470-04_7_additional.jpg   
29446        WOMEN-Sweaters-id_00004473-01_1_front.jpg   
13689       WOMEN-Cardigans-id_00006470-04_1_front.jpg   
13809        WOMEN-Cardigans-id_00007372-02_2_side.jpg   

                                                 caption  token_length  
13691  The lady wears a long-sleeve sweater with pure...           112  
13692  This person is wearing a long-sleeve sweater w...           110  
29446  This lady is wearing a long-sleeve sweater wit...           109  
13689  The sweater this woman wears has long sleeves,...           106  
13809  The person is wearing a long-sleeve sweater wi...           106  


**Vấn đề: Giới hạn độ dài token của CLIP (77 token)**

Mô hình CLIP (clip-vit-base-patch16) có giới hạn tối đa 77 token cho văn bản đầu vào, trong khi dataset DeepFashion chứa 1,744 caption (khoảng 3.96% trong tổng số 44,096 caption) dài hơn 77 token, với độ dài tối đa được báo cáo là 112 token. Điều này dẫn đến việc cắt bớt (truncation) dữ liệu khi sử dụng max_length=77, gây mất thông tin quan trọng.



**Cách giải quyết: Tiền xử lý caption**

Thực hiện tiền xử lý để giữ câu đầu tiên
của các caption dài hơn 77 token, đảm bảo độ dài không vượt quá giới hạn.

Nếu câu đầu tiên vẫn vượt 77 token, áp dụng truncation=True để cắt thêm, giữ thông tin quan trọng nhất.

Kết quả: Giảm mất dữ liệu trong 1,744 caption dài, phù hợp với mean length 50.40, đồng thời tuân thủ giới hạn 77 token của CLIP.



In [11]:
# Hàm tiền xử lý caption (giữ câu đầu tiên và đảm bảo dưới 77 token)
def preprocess_caption(caption):
    inputs = processor.tokenizer(caption, return_tensors="pt", truncation=False)
    if inputs['input_ids'].shape[1] > 77:
        sentences = caption.split('. ')
        first_sentence = sentences[0] + '.' if sentences else caption
        return first_sentence
    return caption

# Áp dụng xử lý và shuffle DataFrame
caption_df['caption'] = caption_df['caption'].apply(preprocess_caption)
caption_df = caption_df.sample(frac=1).reset_index(drop=True)  # shuffle

**Vấn đề: Lỗi RuntimeError: stack expects each tensor to be equal size**

Khi huấn luyện, mô hình gặp lỗi do kích thước tensor không đồng nhất (ví dụ: [50] và [66] cho input_ids) do độ dài caption khác nhau, dẫn đến việc torch.stack không thể thực hiện.



**Cách giải quyết:**Thêm padding và collate_fn: Cấu hình CLIPProcessor với padding="max_length" và max_length=77 để đảm bảo tất cả input_ids và attention_mask có cùng kích thước.

**Triển khai collate_fn tùy chỉnh:** Sử dụng hàm collate_fn để stack các tensor (pixel_values, input_ids, attention_mask) một cách đồng nhất, khắc phục lỗi này và đảm bảo dữ liệu đầu vào phù hợp cho mô hình.



In [12]:
# Chia train/validation theo tỉ lệ 90/10
train_df, val_df = train_test_split(caption_df, test_size=0.1, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


In [13]:
# Tạo Dataset tùy chỉnh
class DeepFashionDataset(Dataset):
    def __init__(self, image_dir, caption_df, processor):
        self.image_dir = image_dir
        self.caption_df = caption_df
        self.processor = processor

    def __len__(self):
        return len(self.caption_df)

    def __getitem__(self, idx):
        row = self.caption_df.iloc[idx]
        image_name = row['image_name']
        caption = row['caption']
        image_path = os.path.join(self.image_dir, image_name)
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at {image_path}")
        image = Image.open(image_path).convert('RGB')
        inputs = self.processor(text=caption, images=image, return_tensors="pt", padding="max_length", max_length=77, truncation=True)
        return {
            'pixel_values': inputs['pixel_values'].squeeze(0),
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0)
        }

# Hàm collate_fn custom
def collate_fn(batch):
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    return {
        'pixel_values': pixel_values,
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

In [14]:
# Tạo dataset và DataLoader
dataset = DeepFashionDataset(image_dir, train_df, processor)
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

val_dataset = DeepFashionDataset(image_dir, val_df, processor)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)
# Kiểm tra một mẫu
sample = dataset[0]
print("Sample keys:", sample.keys())
print("Sample pixel_values shape:", sample['pixel_values'].shape)
print("Sample input_ids shape:", sample['input_ids'].shape)
print("Sample caption:", train_df.iloc[0]['caption'])

Sample keys: dict_keys(['pixel_values', 'input_ids', 'attention_mask'])
Sample pixel_values shape: torch.Size([3, 224, 224])
Sample input_ids shape: torch.Size([77])
Sample caption: The person is wearing a tank tank top with graphic patterns and a three-point pants. The tank top is with cotton fabric and its neckline is suspenders. The pants are with cotton fabric and solid color patterns. The person has neckwear. This person wears a ring. There is an accessory on her wrist.


#3 Fine-tuning CLIP Process


##Finetune all-in-one

In [15]:

class CustomCLIPWithTemperature(nn.Module):
    def __init__(self, pretrained_model="openai/clip-vit-base-patch16", temperature_init=0.07):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(pretrained_model)
        self.temperature = nn.Parameter(torch.tensor(temperature_init))

    def forward(self, pixel_values, input_ids, attention_mask):
        outputs = self.clip(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        image_embeds = F.normalize(outputs.image_embeds, dim=-1)
        text_embeds = F.normalize(outputs.text_embeds, dim=-1)
        logits_per_image = torch.matmul(image_embeds, text_embeds.T) / self.temperature
        logits_per_text = logits_per_image.T
        return logits_per_image, logits_per_text

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
# Định nghĩa thư mục lưu mô hình và processor
model_dir = "/content/drive/MyDrive/finetuned_clip_final"
if os.path.exists(model_dir):
    print("Loading model and processor from saved directory.")
    model = CustomCLIPWithTemperature()
    model.clip = CLIPModel.from_pretrained(model_dir).to(device)
    processor = CLIPProcessor.from_pretrained(model_dir)
else:
    print("Loading pretrained model and processor.")
    model = CustomCLIPWithTemperature().to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")


Using device: cuda
Loading model and processor from saved directory.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [17]:
# Tạo path checkpoint
checkpoint_dir = '/content/drive/MyDrive/finetuned_clip_checkpoints'
checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')

# Kiểm tra và tạo thư mục nếu chưa tồn tại
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print(f"Created checkpoint directory at {checkpoint_dir}")
else:
    print(f"Checkpoint directory already exists at {checkpoint_dir}")

Checkpoint directory already exists at /content/drive/MyDrive/finetuned_clip_checkpoints


In [18]:
optimizer = AdamW(model.parameters(), lr=5e-6)
start_epoch = 0
# Load checkpoint nếu tồn tại
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint.get('epoch', -1) + 1 # tiếp tục từ epoch kế tiếp
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Starting training from scratch")

Resumed from epoch 29


In [19]:
# Hàm huấn luyện với contrastive loss
def train_step(batch):
    pixel_values = batch['pixel_values'].to(device)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    logits_img, logits_txt = model(pixel_values, input_ids, attention_mask)
    labels = torch.arange(logits_img.size(0)).to(device)
    loss_img = F.cross_entropy(logits_img, labels)
    loss_txt = F.cross_entropy(logits_txt, labels)
    return (loss_img + loss_txt) / 2

In [20]:
 #Hàm evaluate Recall@K
 @torch.no_grad()
def evaluate_recall_at_k(model, dataloader, device="cuda", k_values=[1, 5, 10]):
    model.eval()
    image_embeds_list, text_embeds_list = [], []
    for batch in tqdm(dataloader, desc="Evaluating..."):
        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model.clip(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        image_embeds = F.normalize(outputs.image_embeds, dim=-1)
        text_embeds = F.normalize(outputs.text_embeds, dim=-1)
        image_embeds_list.append(image_embeds)
        text_embeds_list.append(text_embeds)

    image_embeds = torch.cat(image_embeds_list)
    text_embeds = torch.cat(text_embeds_list)
    sim_matrix = torch.matmul(text_embeds, image_embeds.T)
    recall_scores = {k: 0 for k in k_values}
    for i in range(sim_matrix.size(0)):
        top_k = torch.topk(sim_matrix[i], max(k_values)).indices
        for k in k_values:
            if i in top_k[:k]:
                recall_scores[k] += 1
    return {f"Recall@{k}": 100. * recall_scores[k] / sim_matrix.size(0) for k in k_values}


In [21]:
max_epochs = 30

# Vòng lặp huấn luyện
for epoch in range(start_epoch, max_epochs):
    print(f"Epoch {epoch + 1}/{max_epochs} started.")
    model.train()
    total_loss = 0

    for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        optimizer.zero_grad()
        loss = train_step(batch)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        total_loss += batch_loss

        # In trạng thái mỗi 100 batch (tuỳ chỉnh)
        if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(train_dataloader):
            print(f"Epoch {epoch + 1} | Batch {batch_idx + 1}/{len(train_dataloader)} | Batch Loss: {batch_loss:.4f}")

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")

    # Lưu checkpoint sau mỗi epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at end of epoch {epoch + 1}")

    # Đánh giá
    recall = evaluate_recall_at_k(model, val_dataloader, device)
    print(f"Validation result after epoch {epoch + 1}:")
    for k, v in recall.items():
        print(f"  Recall@{k}: {v:.2f}%")

    # Lưu model
    model.clip.save_pretrained(model_dir)
    processor.save_pretrained(model_dir)
    print(f"Model and processor saved to {model_dir}")

Epoch 30/30 started.


Epoch 30:   4%|▍         | 100/2394 [01:43<37:10,  1.03it/s]

Epoch 30 | Batch 100/2394 | Batch Loss: 0.0040


Epoch 30:   8%|▊         | 200/2394 [03:27<36:10,  1.01it/s]

Epoch 30 | Batch 200/2394 | Batch Loss: 0.0002


...

Epoch 30: 100%|██████████| 2394/2394 [40:51<00:00,  1.02s/it]

Epoch 30 | Batch 2394/2394 | Batch Loss: 0.0000
Epoch 30 completed. Average Loss: 0.0370


Checkpoint saved at end of epoch 30


Evaluating...: 100%|██████████| 266/266 [02:08<00:00,  2.07it/s]


Validation result after epoch 30:
  Recall@Recall@1: 34.45%
  Recall@Recall@5: 72.10%
  Recall@Recall@10: 86.49%
Model and processor saved to /content/drive/MyDrive/finetuned_clip_final


#Push finetuned model to hugging face


In [22]:
from google.colab import drive
import torch
from transformers import CLIPModel, CLIPProcessor
from torch.optim import AdamW
from tqdm import tqdm
from PIL import Image
import os
import json
import numpy as np
from sklearn.cluster import KMeans
from pathlib import Path
import gradio as gr
from pyngrok import ngrok
from huggingface_hub import HfApi, Repository

#from clip import CLIPSearcher
#from db import SearchMechanism

In [23]:
!pip install huggingface_hub

**Cấu hình đầu vào & xác thực**

In [24]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `PushModelFromColab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-au

**cấu hình đường dẫn & repo_id**

In [25]:
# Cấu hình
local_model_dir = "/content/finetuned_clip_final"
source_dir = "/content/drive/MyDrive/finetuned_clip_final"
repo_id = "anhquanlam/clip-finetuned-deepfashion"  # Thay bằng repo của bạn


**Tải model đã fine-tune và lưu chuẩn bị push**

In [26]:
# Load model & processor từ drive (hoặc output của training)
model = CLIPModel.from_pretrained(source_dir)
processor = CLIPProcessor.from_pretrained(source_dir)

# Lưu lại vào /content để đẩy
model.save_pretrained(local_model_dir)
processor.save_pretrained(local_model_dir)


[]

**Push lên Hugging Face Hub**

 lần đầu

In [ ]:
# Push cả model và processor
model.push_to_hub(repo_id, private=False, commit_message="Initial fine-tuned CLIP model upload")
processor.push_to_hub(repo_id, commit_message="Initial processor upload")
print(f"Model and processor pushed to: https://huggingface.co/{repo_id}")


README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/anhquanlam/CDXLAV-clip-finetuned-model/commit/518036a6bc78750f052d6d4bcc6cc83371fa0e53', commit_message='Upload processor', commit_description='', oid='518036a6bc78750f052d6d4bcc6cc83371fa0e53', pr_url=None, repo_url=RepoUrl('https://huggingface.co/anhquanlam/CDXLAV-clip-finetuned-model', endpoint='https://huggingface.co', repo_type='model', repo_id='anhquanlam/CDXLAV-clip-finetuned-model'), pr_revision=None, pr_num=None)

**Nếu cập nhật lại sau khi fine-tune thêm, chỉ cần chạy lại bước này**

In [27]:
# Push cập nhật (không cần lưu lại nếu đã fine-tune tại local_model_dir)
model.push_to_hub(repo_id, commit_message="Update: fine-tuned CLIP model V5 (30 epochs)")
processor.push_to_hub(repo_id, commit_message="Update: processor for V5 (30 epochs)")


README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/anhquanlam/clip-finetuned-deepfashion/commit/0f7dc917c7346de2f941fbe324cd72aeaed01ae5', commit_message='Update: processor for V5 (30 epochs)', commit_description='', oid='0f7dc917c7346de2f941fbe324cd72aeaed01ae5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/anhquanlam/clip-finetuned-deepfashion', endpoint='https://huggingface.co', repo_type='model', repo_id='anhquanlam/clip-finetuned-deepfashion'), pr_revision=None, pr_num=None)